# Лабораторная работа №7

In [21]:
import numpy as np
from tabulate import tabulate
from scipy import stats
from math import log10
import sys

## Постановка задачи
Сгенерировать выборку объёмом 100 элементов для нормального распределения $N(x,0,1)$. По сгенерированной выборке оценить параметры $\mu$ и $\sigma$ нормального закона методом максимального правдоподобия. В качестве основной гипотезы $H_0$ будем считать, что сгенерированное распределение имеет вид $N(x,\hat{\mu},\hat{\sigma})$. Проверить основную гипотезу, используя критерий согласия χ2. В качестве уровня значимости взять $\alpha = 0.05$. Привести таблицу вычислений χ2.

In [108]:
alpha = 0.05
p = 1 - alpha
size = 20
k = 5
l, r = -2, 2

## Реализация

In [113]:
np.random.seed(7)
sys.stdout = open("hypothesis.txt", "w")

dist = np.random.uniform(-2, 2, size)
mu = np.mean(dist)
sigma = np.std(dist)

print("Maximum likelihood estimation:")
print('mu =', np.around(mu, decimals=2), ' ', 'sigma =', np.around(sigma, decimals=2))
print()

print("Consent criterion X^2:")
delta = np.linspace(-1.01, 1.56, num=k - 1)
value = stats.chi2.ppf(p, k - 1)
array = np.array([stats.norm.cdf(delta[0])])
q = np.array([len(dist[dist <= delta[0]])])

for i in range(0, len(delta) - 1):
    new_ar = stats.norm.cdf(delta[i + 1]) - stats.norm.cdf(delta[i])
    array = np.append(array, new_ar)
    q = np.append(q, len(dist[(dist <= delta[i + 1]) & (dist >= delta[i])]))


array = np.append(array, 1 - stats.norm.cdf(delta[-1]))
q = np.append(q, len(dist[dist >= delta[-1]]))
result = np.divide(np.multiply((q - size * array), (q - size * array)), array * size)

table_headers = ["i", '$Delta_i$', "$n_i$", "$p_i$", "$np_i$", "$n_i-np_i$", "$frac{(n_i-np_i)^2}{np_i}$"]
rows = []

for i in range(0, len(q)):
    if i == 0:
        boarders = ['-infity', np.around(delta[0], decimals=2)]
    elif i == len(q) - 1:
        boarders = [np.around(delta[-1], decimals=2), 'infity']
    else:
        boarders = [np.around(delta[i - 1], decimals=2), np.around(delta[i], decimals=2)]

    rows.append([i + 1, boarders, q[i], np.around(array[i], decimals=4), np.around(array[i] * size, decimals=2),
                 np.around(q[i] - size * array[i], decimals=2), np.around(result[i], decimals=2)])


rows.append([len(q), "-", np.sum(q), np.around(np.sum(array), decimals=4),
             np.around(np.sum(array * size), decimals=2),
             -np.around(np.sum(q - size * array), decimals=2),
             np.around(np.sum(result), decimals=2)])


print(tabulate(rows, table_headers))